In [171]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score,recall_score, confusion_matrix, classification_report,accuracy_score, f1_score
import numpy as np
import pandas as pd
import random
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve , auc
from sklearn.metrics.cluster import fowlkes_mallows_score as gmean
import statistics as stat

In [172]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [173]:
df=pd.read_csv('/content/drive/MyDrive/yeast-0-5-6-7-9_vs_4.csv')
df

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,Class
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,negative
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,negative
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,negative
3,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,negative
4,0.50,0.54,0.48,0.65,0.5,0.0,0.53,0.22,negative
...,...,...,...,...,...,...,...,...,...
523,0.81,0.90,0.39,0.41,0.5,0.0,0.53,0.22,positive
524,0.83,0.53,0.52,0.20,0.5,0.0,0.48,0.25,positive
525,0.78,0.48,0.47,0.30,0.5,0.0,0.53,0.47,positive
526,0.72,0.56,0.40,0.26,0.5,0.0,0.51,0.26,positive


In [174]:
df["Class"].value_counts()
class_label=df['Class']
class_label

0      negative
1      negative
2      negative
3      negative
4      negative
         ...   
523    positive
524    positive
525    positive
526    positive
527    positive
Name: Class, Length: 528, dtype: object

In [175]:
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df.head()


,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,Class
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,0
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,0
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,0
3,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,0
4,0.50,0.54,0.48,0.65,0.5,0.0,0.53,0.22,0


In [176]:
minor=df[df["Class"]==1]
major=df[df["Class"]==0]
minor=minor.drop(["Class"], axis=1)
minority=minor.to_numpy()
minor.shape
major.shape
diff=major.shape[0]-minor.shape[0]
r=diff
loop=diff/minor.shape[0]
if(diff%minor.shape[0]==0):
    iter=int(diff/minor.shape[0])
else:
  iter=int(diff/minor.shape[0])+1
minor.shape


(51, 8)

In [177]:
for id in range(r):
  if(diff>0):
    distances = np.linalg.norm(minority - minority[id], axis=1)  #calculating the distances to other instances
    k = 4
    nearest_neighbor_ids = distances.argsort()[:k] #finding 3 nearest neighbours n1,n2 and n3
    n1=minority[nearest_neighbor_ids[1]]
    n2=minority[nearest_neighbor_ids[2]]
    n3=minority[nearest_neighbor_ids[3]]
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    w=random.uniform(0.5,1)
    velocity= w+r1*(n1-n3) + r2*(n1-n2)         #velocity vector
    new_minor=minority[id]+velocity             #position update
    d=minor.shape[1] 
    minB=minor.min().to_numpy()
    maxB=minor.max().to_numpy()
    for i in range(d):
      if(new_minor[i]<minB[i]):
        new_minor[i]=minB[i];
      if(new_minor[i]>maxB[i]):
        new_minor[i]=maxB[i];
    minority=np.append(minority,new_minor)
    minority=np.reshape(minority,(-1, minor.shape[1]))
    diff=diff-1

minority.shape

(477, 8)

In [178]:
df_minor = pd.DataFrame(minority, columns =['Mcg','Gvh','Alm','Mit','Erl','Pox','Vac','Nuc'])
df_minor

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc
0,0.86,0.68,0.39,0.24,0.5,0.0,0.54,0.22
1,0.79,0.57,0.41,0.28,0.5,0.0,0.59,0.22
2,0.74,0.75,0.58,0.47,0.5,0.0,0.40,0.35
3,0.69,0.61,0.38,0.27,0.5,0.0,0.52,0.22
4,0.54,0.42,0.41,0.16,0.5,0.0,0.52,0.22
...,...,...,...,...,...,...,...,...
472,0.97,0.90,0.60,0.57,1.0,0.0,0.61,0.47
473,0.97,0.90,0.60,0.57,1.0,0.0,0.61,0.47
474,0.97,0.90,0.60,0.57,1.0,0.0,0.61,0.47
475,0.97,0.90,0.60,0.57,1.0,0.0,0.61,0.47


In [179]:
dup=[]
for i in range(df_minor.shape[0]):
  dup.append(1)
Class=np.array(dup)
Class=np.reshape(Class,(-1,1))
df_minor_class=pd.DataFrame(Class,columns=['Class'])
df_minor['Class']=df_minor_class
df_minor

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,Class
0,0.86,0.68,0.39,0.24,0.5,0.0,0.54,0.22,1
1,0.79,0.57,0.41,0.28,0.5,0.0,0.59,0.22,1
2,0.74,0.75,0.58,0.47,0.5,0.0,0.40,0.35,1
3,0.69,0.61,0.38,0.27,0.5,0.0,0.52,0.22,1
4,0.54,0.42,0.41,0.16,0.5,0.0,0.52,0.22,1
...,...,...,...,...,...,...,...,...,...
472,0.97,0.90,0.60,0.57,1.0,0.0,0.61,0.47,1
473,0.97,0.90,0.60,0.57,1.0,0.0,0.61,0.47,1
474,0.97,0.90,0.60,0.57,1.0,0.0,0.61,0.47,1
475,0.97,0.90,0.60,0.57,1.0,0.0,0.61,0.47,1


In [180]:
majority=major.to_numpy()
majority=np.reshape(majority,(-1,major.shape[1]))
majority
df_major = pd.DataFrame(majority, columns = ['Mcg','Gvh','Alm','Mit','Erl','Pox','Vac','Nuc','Class'])
df_major

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,Class
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,0.0
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,0.0
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,0.0
3,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,0.0
4,0.50,0.54,0.48,0.65,0.5,0.0,0.53,0.22,0.0
...,...,...,...,...,...,...,...,...,...
472,0.78,0.74,0.58,0.25,1.0,0.0,0.53,0.22,0.0
473,0.77,0.80,0.51,0.40,1.0,0.0,0.54,0.22,0.0
474,0.85,0.56,0.33,0.38,1.0,0.0,0.55,0.25,0.0
475,0.86,0.92,0.50,0.37,1.0,0.0,0.53,0.32,0.0


In [181]:
balanced_df=pd.concat([df_minor,df_major],axis=0,ignore_index=True)
balanced_df

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,Class
0,0.86,0.68,0.39,0.24,0.5,0.0,0.54,0.22,1.0
1,0.79,0.57,0.41,0.28,0.5,0.0,0.59,0.22,1.0
2,0.74,0.75,0.58,0.47,0.5,0.0,0.40,0.35,1.0
3,0.69,0.61,0.38,0.27,0.5,0.0,0.52,0.22,1.0
4,0.54,0.42,0.41,0.16,0.5,0.0,0.52,0.22,1.0
...,...,...,...,...,...,...,...,...,...
949,0.78,0.74,0.58,0.25,1.0,0.0,0.53,0.22,0.0
950,0.77,0.80,0.51,0.40,1.0,0.0,0.54,0.22,0.0
951,0.85,0.56,0.33,0.38,1.0,0.0,0.55,0.25,0.0
952,0.86,0.92,0.50,0.37,1.0,0.0,0.53,0.32,0.0


In [182]:

from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)
x_b=balanced_df.drop(['Class'],axis='columns')
y_b=balanced_df.Class




In [183]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
model_b_s=SVC()

model_b_k=KNeighborsClassifier(n_neighbors=3)


model_b_d=DecisionTreeClassifier(random_state=0)

In [184]:
#precision
precision_s=[]
f1_s=[]
recall_s=[]
accuracy_s=[]
Auc_s=[]
Gmean_s=[]

precision_k=[]
f1_k=[]
recall_k=[]
accuracy_k=[]
Auc_k=[]
Gmean_k=[]

precision_d=[]
f1_d=[]
recall_d=[]
accuracy_d=[]
Auc_d=[]
Gmean_d=[]
for train_index_b,test_index_b in kf.split(x_b,y_b):
  x_train_b,x_test_b=x_b.iloc[train_index_b],x_b.iloc[test_index_b]
  y_train_b,y_test_b=y_b.iloc[train_index_b],y_b.iloc[test_index_b]
  model_b_s.fit(x_train_b,y_train_b)
  prediction_b_s = model_b_s.predict(x_test_b)

  model_b_k.fit(x_train_b,y_train_b)
  prediction_b_k = model_b_k.predict(x_test_b)

  model_b_d.fit(x_train_b,y_train_b)
  prediction_b_d = model_b_d.predict(x_test_b)

  precision_s.append(precision_score(y_test_b,prediction_b_s))
  precision_k.append(precision_score(y_test_b,prediction_b_k))
  precision_d.append(precision_score(y_test_b,prediction_b_d))

  recall_s.append(recall_score(y_test_b,prediction_b_s))
  recall_k.append(recall_score(y_test_b,prediction_b_k))
  recall_d.append(recall_score(y_test_b,prediction_b_d))

  f1_s.append(f1_score(y_test_b,prediction_b_s))
  f1_k.append(f1_score(y_test_b,prediction_b_k))
  f1_d.append(f1_score(y_test_b,prediction_b_d))

  accuracy_s.append(accuracy_score(y_test_b,prediction_b_s))
  accuracy_k.append(accuracy_score(y_test_b,prediction_b_k))
  accuracy_d.append(accuracy_score(y_test_b,prediction_b_d))


  s_fpr_b,s_tpr_b,threshold_s_b=roc_curve(y_test_b,prediction_b_s)
  k_fpr_b,k_tpr_b,threshold_k_b=roc_curve(y_test_b,prediction_b_k)
  d_fpr_b,d_tpr_b,threshold_d_b=roc_curve(y_test_b,prediction_b_d)


  Auc_s.append(auc(s_fpr_b,s_tpr_b))
  Auc_k.append(auc(k_fpr_b,k_tpr_b))
  Auc_d.append(auc(d_fpr_b,d_tpr_b))


  Gmean_s.append(gmean(y_test_b,prediction_b_s))
  Gmean_k.append(gmean(y_test_b,prediction_b_k))
  Gmean_d.append(gmean(y_test_b,prediction_b_d))


print("-------SVM--------")
print("Gmean_SVM:",Gmean_s)
print("AUC_SVM:",Auc_s)
print("f1_SVM:",f1_s)
print("precision_SVM:",precision_s)
print("recall_SVM:",recall_s)

print("mean of Gmean_s:",stat.mean(Gmean_s))
print("mean of AUC_s:",stat.mean(Auc_s))
print("mean of f1_s:",stat.mean(f1_s))
print("mean of precision_s:",stat.mean(precision_s))
print("mean of recall_s:",stat.mean(recall_s))

print("standard deviation of Gmean_s:",stat.stdev(Gmean_s))
print("standard deviation of AUC_s:",stat.stdev(Auc_s))
print("standard deviation of f1_s:",stat.stdev(f1_s))
print("standard deviation of precision_s:",stat.stdev(precision_s))
print("standard deviation of recall_s:",stat.stdev(recall_s))

print("\n")
print("-------KNN--------")
print("Gmean_KNN:",Gmean_k)
print("AUC_KNN:",Auc_k)
print("f1_KNN:",f1_k)
print("precision_KNN:",precision_k)
print("recall_KNN:",recall_k)

print("mean of Gmean_k:",stat.mean(Gmean_k))
print("mean of AUC_k:",stat.mean(Auc_k))
print("mean of f1_k:",stat.mean(f1_k))
print("mean of precision_k:",stat.mean(precision_k))
print("mean of recall_k:",stat.mean(recall_k))

print("standard deviation of Gmean_k:",stat.stdev(Gmean_k))
print("standard deviation of AUC_k:",stat.stdev(Auc_k))
print("standard deviation of f1_k:",stat.stdev(f1_k))
print("standard deviation of precision_k:",stat.stdev(precision_k))
print("standard deviation of recall_k:",stat.stdev(recall_k))

print("\n")
print("-------DECISION TREE--------")
print("Gmean_DT:",Gmean_d)
print("AUC_DT:",Auc_d)
print("f1_DT:",f1_d)
print("precision_DT:",precision_d)
print("recall_DT:",recall_d)

print("mean of Gmean_d:",stat.mean(Gmean_d))
print("mean of AUC_d:",stat.mean(Auc_d))
print("mean of f1_d:",stat.mean(f1_d))
print("mean of precision_d:",stat.mean(precision_d))
print("mean of recall_d:",stat.mean(recall_d))

print("standard deviation of Gmean_d:",stat.stdev(Gmean_d))
print("standard deviation of AUC_d:",stat.stdev(Auc_d))
print("standard deviation of f1_d:",stat.stdev(f1_d))
print("standard deviation of precision_d:",stat.stdev(precision_d))
print("standard deviation of recall_d:",stat.stdev(recall_d))



-------SVM--------
Gmean_SVM: [0.6584623673788136, 1.0, 1.0, 1.0, 0.777290516621462]
AUC_SVM: [0.734375, 1.0, 1.0, 1.0, 0.868421052631579]
f1_SVM: [0.6382978723404256, 1.0, 1.0, 1.0, 0.8837209302325582]
precision_SVM: [1.0, 1.0, 1.0, 1.0, 0.7916666666666666]
recall_SVM: [0.46875, 1.0, 1.0, 1.0, 1.0]
mean of Gmean_s: 0.8871505768000552
mean of AUC_s: 0.9205592105263158
mean of f1_s: 0.9044037605145967
mean of precision_s: 0.9583333333333334
mean of recall_s: 0.89375
standard deviation of Gmean_s: 0.1601347143036282
standard deviation of AUC_s: 0.1186543982060617
standard deviation of f1_s: 0.1570478002080034
standard deviation of precision_s: 0.09316949906249125
standard deviation of recall_s: 0.23758222260935266


-------KNN--------
Gmean_KNN: [0.6584623673788136, 0.9586060105102576, 0.9790604979210692, 0.9687638992999191, 0.7510940891927954]
AUC_KNN: [0.734375, 0.9789473684210526, 0.9895833333333333, 0.984375, 0.8473684210526315]
f1_KNN: [0.6382978723404256, 0.9795918367346939, 0.9895

In [187]:
list=['yeast05679vs4',stat.mean(Auc_k),stat.stdev(Auc_k),stat.mean(Gmean_k),stat.stdev(Gmean_k),stat.mean(Auc_d),stat.stdev(Auc_d),stat.mean(Gmean_d),stat.stdev(Gmean_d),stat.mean(Auc_s),stat.stdev(Auc_s),stat.mean(Gmean_s),stat.stdev(Gmean_s)]
list

df_excel = pd.read_excel('/content/drive/MyDrive/pso_1.xlsx')
df_excel

,Unnamed: 0,Unnamed: 0.1,KNN,Unnamed: 2,Unnamed: 3,Unnamed: 4,Decision Tree,Unnamed: 6,Unnamed: 7,Unnamed: 8,SVM,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,0,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN
1,1,Dataset,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev
2,2,ecoli0137vs26,0.959495,0.040136,0.92449,0.072456,0.959798,0.027859,0.923228,0.052104,0.946633,0.061364,0.905228,0.105836
3,3,shuttle0vs4,0.999414,0.000802,0.998826,0.001607,1,0,1,0,0.998828,0.000655,0.997651,0.001313
4,4,yeast1vs7,0.953475,0.075796,0.923035,0.116982,0.938317,0.063328,0.89309,0.093762,0.963953,0.080602,0.94367,0.125957
5,5,shuttle2vs4,1,0,1,0,1,0,1,0,1,0,1,0
6,6,glass016vs2,0.925714,0.102718,0.885581,0.143202,0.885714,0.108327,0.827411,0.127491,0.591429,0.180871,0.723895,0.066047
7,7,glass016vs5,0.885714,0.137396,0.845013,0.164663,0.88,0.149352,0.84569,0.173688,0.951429,0.065153,0.914833,0.107251
8,8,pageblocks13vs4,0.921157,0.041699,0.858736,0.064528,0.966279,0.052842,0.939967,0.090831,0.781563,0.023842,0.673539,0.031343


In [188]:
df_excel.loc[len(df_excel)]=list
df_excel.to_excel('/content/drive/MyDrive/pso_1.xlsx')